# Two-Sample and ANOVA Tests

# 1. Performing t-tests

<b>1.1 Hypothesis testing workflow</b>

You've seen the hypothesis testing workflow for the one sample case where you compared a sample mean to a hypothesized value, and the two sample case where you compared two sample means. In both cases, the workflow shares common steps.

Place the hypothesis testing workflow steps in order from first to last.

![Alt text](Hypothesis%20testing%20workflow.png)

Regardless of the type of hypothesis test you are performing, it will have a workflow that follows this format.